In [2]:
import pandas as pd
import pgeocode
import numpy as np
import time
# import tensorflow as tf
# import keras_tuner


In [3]:
# URLs

#zipcode url
zip_url = 'https://raw.githubusercontent.com/symerio/postal-codes-data/master/data/geonames/US.txt'
zipcode_raw_df = pd.read_csv(zip_url, header=None, sep='\t')
zipcode_df = zipcode_raw_df[[1,9,10]].rename(columns={1:'zip',
                                                        9: 'lat',
                                                        10:'lon'})
zipcode_df.drop_duplicates(inplace=True)
zipcode_df.nunique()
# crime url

# water url

# rent url

zip    41481
lat    33585
lon    35876
dtype: int64

In [4]:
# load in crime data and store in df
crime_csv = 'Resources/crime_data.csv'
crime_raw_df = pd.read_csv(crime_csv)
crime_raw_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [5]:
# remove rows where lat or lon have 0.0 values
crime_df = crime_raw_df.loc[(crime_raw_df['LAT'] != 0.0) & (crime_raw_df['LON'] != 0.0)]
print(f'{len(crime_df)} Eligible LA Crime Reports Found!\n')

# retrieve min and max values of lat and lon for LA crime data
min_lat = min(crime_df['LAT'])
print(f'Minimum Latitude: {min_lat}')
max_lat = max(crime_df['LAT'])
print(f'Maximum Latitude: {max_lat}')
min_lon = min(crime_df['LON'])
print(f'Minimum Longitude: {min_lon}')
max_lon = max(crime_df['LON'])
print(f'Maximum Latitude: {max_lon}')

# use min and max lat and lon values in crime data to reduce iterable size of zipcode df to rough LA region
zipcode_df = zipcode_df.loc[((zipcode_df['lat'] >= min_lat) & (zipcode_df['lat'] <= max_lat)) & ((zipcode_df['lon'] >= min_lon) & (zipcode_df['lon'] <= max_lon))]
print(f'\n\n{len(zipcode_df)} Eligible LA Zipcodes Found!\n')
zipcode_arr = zipcode_df.values.astype('float32')
print(zipcode_arr[0])
# print(f"Min Zipcode: {min(zipcode_df['zip'].values).astype(int)}")
# print(f"Max Zipcode: {max(zipcode_df['zip'].values).astype(int)}")

# zipcode_arr = zipcode_df.values[:,0].astype(int)
# coord_arr =  zipcode_df.values[:,1:3].astype('float32')
# print(zipcode_arr[0:5])
# print(coord_arr[0:5])

701541 Eligible LA Crime Reports Found!

Minimum Latitude: 33.7061
Maximum Latitude: 34.3343
Minimum Longitude: -118.6676
Maximum Latitude: -118.1554


329 Eligible LA Zipcodes Found!

[ 9.000100e+04  3.397310e+01 -1.182479e+02]


In [8]:
def distance(x,y):
    # x1 = x[0]
    # y1 = x[1]
    # x2 = y[0]
    # y2 = y[1]

    return ((x[0]-y[0])**2 + (x[1]-y[1])**2)**(1/2)

In [9]:
distance([1,2],[3,4])

2.8284271247461903

In [10]:
# making arrays and sending coordinates through get_zip function
def get_zip(test_coords):
    dists = []
    start = time.perf_counter()
    for i in coord_arr:
        dists.append(distance(test_coords, i))
    
    return zip_arr[np.argmin(dists)][0]
    
# zips = []

# start = time.time()
# zips.append(get_zip(test_coords))
# print(time.time() - start)


In [13]:
coord_arr = zipcode_df[['lat','lon']].values
zip_arr = zipcode_df[['zip']].values
test_coords = [34.11921572890567, -118.30030766931306]

get_zip(test_coords)

90027

In [12]:
crime_coords = crime_df[['LAT','LON']]
crime_coords = crime_coords.values.astype('float64')
crime_zips = []
for i in crime_coords:
    crime_zips.append(get_zip(i))

# crime_df['zipcode'] = np.apply_along_axis(get_zip, 1, crime_coords)

In [14]:
crime_df['zipcode'] = crime_zips
crime_df.to_csv('Resources/crime_wzip.csv')

/Users/max/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
# load in water quality data and store in df
water_csv = 'Resources/water_qual_lab_results.csv'
water_df = pd.read_csv(water_csv)
water_df.head()

,station_id,station_name,full_station_name,station_number,station_type,latitude,longitude,status,county_name,sample_code,sample_date,sample_depth,sample_depth_units,parameter,result,reporting_limit,units,method_name
0,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Conductance,3480.0,1.0,uS/cm,EPA 120.1
1,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Boron,7.7,0.1,mg/L,"Std Method 4500-B, C"
2,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Calcium,68.0,1.0,mg/L,EPA 215.2
3,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Chloride,758.0,0.1,mg/L,"Std Method 4500-Cl, B"
4,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.5617,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Magnesium,59.0,0.1,mg/L,"Std Method 3500-Mg, E"


In [ ]:
# convert water_df lat,lng to zip codes
# filter out unnecessary zip codes from water_df (only values within min and max of crime dataset)



In [ ]:
# load via api

In [ ]:
 # clean with pandas

In [ ]:
# load to postgres SQL

In [ ]:
# query specific columns from SQL

In [ ]:
# run models

In [ ]:
# save models somewhere (Google Drive?)